<a href="https://colab.research.google.com/github/ductt-1167/QuantizationTutorial/blob/main/TFlite_MNIST_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thử nghiệm sử dụng TFlite
Với mô hình, trong đó mô hình chứa cả LSTM layer, trên tập dataset MNIST 

# Build MNIST LSTM model

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
#@title Build model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28, 28), name='input'),
    tf.keras.layers.LSTM(20, time_major=False, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 20)            3920      
                                                                 
 flatten (Flatten)           (None, 560)               0         
                                                                 
 output (Dense)              (None, 10)                5610      
                                                                 
Total params: 9,530
Trainable params: 9,530
Non-trainable params: 0
_________________________________________________________________


In [4]:
#@title load MNIST dataset and training model
# Load MNIST dataset.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

print(x_train.shape, type(x_train))

# fit model
EPOCHS=50
BATCH_SIZE=64
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
model.evaluate(x_test, y_test, verbose=0)

(60000, 28, 28) <class 'numpy.ndarray'>
Epoch 1/50
938/938 [==============================] - 14s 8ms/step - loss: 0.4231 - accuracy: 0.8767
Epoch 2/50
938/938 [==============================] - 7s 8ms/step - loss: 0.1529 - accuracy: 0.9551
Epoch 3/50
938/938 [==============================] - 7s 8ms/step - loss: 0.1142 - accuracy: 0.9659
Epoch 4/50
938/938 [==============================] - 7s 8ms/step - loss: 0.0939 - accuracy: 0.9715
Epoch 5/50
938/938 [==============================] - 7s 8ms/step - loss: 0.0808 - accuracy: 0.9755
Epoch 6/50
938/938 [==============================] - 10s 11ms/step - loss: 0.0712 - accuracy: 0.9786
Epoch 7/50
938/938 [==============================] - 10s 10ms/step - loss: 0.0645 - accuracy: 0.9798
Epoch 8/50
938/938 [==============================] - 7s 8ms/step - loss: 0.0583 - accuracy: 0.9816
Epoch 9/50
938/938 [==============================] - 7s 8ms/step - loss: 0.0535 - accuracy: 0.9834
Epoch 10/50
938/938 [==============================] - 

[0.06861316412687302, 0.9839000105857849]

# Convert model to TFLite model

In [5]:
run_model = tf.function(lambda x: model(x))
# This is important, let's FIX the input size.
BATCH_SIZE = 1  
STEPS = 28
INPUT_SIZE = 28
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "/content/keras_lstm"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

# convert
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /content/keras_lstm/assets


INFO:tensorflow:Assets written to: /content/keras_lstm/assets


# Check the converted TensorFlow Lite model

In [6]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

for i in range(TEST_CASES):
  expected = model.predict(x_test[i:i+1])
  interpreter.set_tensor(input_details[0]["index"], x_test[i:i+1, :, :])
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result, decimal=5)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
